In [98]:
import torch.nn.functional as F
from torch.autograd import Variable
import copy
from collections import Counter
import torch 
import numpy as np
import torch.nn as nn

In [99]:
def read_data(path):
    f = open(path, "r")
    lines = f.readlines()
    f.close()
    data = []
    for l in lines:
        labelSplit = l.replace('\n','').split(' ', 1)
        data.append([labelSplit[0], [word.lower() for word in labelSplit[1].split()]])
    return data

data = read_data('./questions.txt')

In [100]:
def remove_stop_words(data, path):
    stop_words = []
    with open(path) as f:
        stop_words = [word for line in f for word in line.split(",")]
    data_without_stop_words = []
    for k, v in data:
        words = [t for t in v if t not in stop_words]
        data_without_stop_words.append((k, words))
    return data_without_stop_words

data = remove_stop_words(data, './stop_words.txt')

In [101]:
def get_labels(data):
    _labels = []
    for k,v in data:
        _labels.append(k)   
    _unique_label = list(set(_labels))
    _unique_label_dict = {}
    for k,v in enumerate(_unique_label):
        _unique_label_dict[v] = k
    return _unique_label_dict

labels = get_labels(data)

In [102]:
def append_labels(data, labels):        
    cleaned_data = []
    for k,v in data:
        cleaned_data.append((labels[k],v))
        
    return np.array(cleaned_data)

data = append_labels(data, labels)

In [103]:
def create_indexed_vocab(data):
    vocab = []
    for _, sent in data:
        for word in sent:
            vocab.append(word)
    count = Counter(vocab)
    count = {w : count[w] for w in count if count[w] >= 2}
    vocab = []
    for k, v in count.items():
        vocab.append(k)
    indexed_vocab = {word: idx for idx, word in enumerate(vocab)}
    return indexed_vocab
def create_vocab(data):
    total_words_orig = []
    for k,sent in data:
        for word in sent:
            total_words_orig.append(word)
    total_words = list(set(total_words_orig))
    total_words_str = ' '.join(total_words)
    vocab = set(total_words_str.split()) 
    word2idx = {word: idx for idx, word in enumerate(vocab)} # create word index
    return word2idx
word2idx=create_vocab(data)
word2idx['#UNK#'] = len(word2idx)
indexed_vocab = create_indexed_vocab(data)
indexed_vocab['#UNK#'] = len(indexed_vocab)

In [104]:
def load_glove_embeddings(path, indexed_vocab, embedding_dim=300):
    with open(path) as f:
        embeddings = np.zeros((len(indexed_vocab), embedding_dim))
        for line in f.readlines():
            values = line.split()
            word = values[0]
            index = indexed_vocab.get(word)
            if index:
                vector = np.array(values[1:], dtype='float32')
                embeddings[index] = vector
        return torch.from_numpy(embeddings).float()

glove_random = load_glove_embeddings('./glove.small.txt', indexed_vocab)
glove_pre = load_glove_embeddings('./glove.small.txt', word2idx)
embeddings_random = nn.Embedding(glove_random.size(0), glove_random.size(1))
embeddings_pretrained = nn.Embedding.from_pretrained(glove_pre, freeze=True)

In [105]:
print(embeddings_random)
print(embeddings_pretrained)

Embedding(3376, 300)
Embedding(8570, 300)


In [106]:
print(glove_random.size())
print(glove_pre.size())

torch.Size([3376, 300])
torch.Size([8570, 300])


In [107]:
def split_train_test(data, test_ratio):
    data_copy = copy.deepcopy(data)
    np.random.shuffle(data_copy)
    test_set_size = int(len(data) * test_ratio)
    test = data_copy[:test_set_size]
    train = data_copy[test_set_size:]
    return train, test

train, test = split_train_test(data, 0.2)

In [144]:
class BOWClassifier(nn.Module):
    def __init__(self,input_size,hidden_size, num_labels):
        super(BOWClassifier, self).__init__()
        self.input_size = input_size
        self.hidden_size  = hidden_size
        self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(self.input_size, num_labels)
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, x):
        hidden = self.fc1(x)
        relu = self.relu(hidden)
        output = self.fc2(x)
        #output = self.sigmoid(output)
        return output

In [145]:
def make_bow_vector(sentence, indexed_vocab):
    pt_tensor= torch.zeros(300, dtype=torch.long)
    count = 0
    for word in sentence:
        count += 1
        if word in indexed_vocab:
            pt_tensor = torch.add(pt_tensor, embeddings_pretrained(torch.LongTensor([indexed_vocab[word]]))[0])
        else:
            pt_tensor = torch.add(pt_tensor, embeddings_pretrained(torch.LongTensor([indexed_vocab['#UNK']]))[0])
    pt_tensor=torch.div(pt_tensor, count)
    return pt_tensor

def get_bow_rep(data):
    bow_data = []
    for label, sent in data:
        bow_data.append(make_bow_vector(sent, word2idx))
    return torch.stack(bow_data)
        
training_set = get_bow_rep(train)

In [146]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size = 300
hidden_size = 100
num_classes = 51
num_epochs = 50
learning_rate = 0.0001

model = BOWClassifier(input_size, hidden_size, num_classes).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

In [147]:
for epoch in range(num_epochs):
    permutation = torch.randperm(len(train))
    i = 0
    for label, question in train[permutation]:
        i += 1
        optimizer.zero_grad()
        bow_vec = make_bow_vector(question, word2idx)
        bow_vec = bow_vec.reshape(-1, 300).to(device)
        label = torch.LongTensor([label])
        label = label.to(device)
        output = model(bow_vec)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Epoch [1/50], Loss: 4.0518
Epoch [1/50], Loss: 3.7560
Epoch [1/50], Loss: 3.7225
Epoch [1/50], Loss: 3.6000
Epoch [1/50], Loss: 3.8785
Epoch [1/50], Loss: 3.5043
Epoch [1/50], Loss: 3.7926
Epoch [1/50], Loss: 3.4167
Epoch [1/50], Loss: 3.6006
Epoch [1/50], Loss: 3.2649
Epoch [1/50], Loss: 3.3179
Epoch [1/50], Loss: 3.2121
Epoch [1/50], Loss: 3.1552
Epoch [1/50], Loss: 3.2205
Epoch [1/50], Loss: 3.3187
Epoch [1/50], Loss: 4.6764
Epoch [1/50], Loss: 3.6325
Epoch [1/50], Loss: 3.1006
Epoch [1/50], Loss: 2.6358
Epoch [1/50], Loss: 3.6131
Epoch [1/50], Loss: 2.5636
Epoch [1/50], Loss: 3.0848
Epoch [1/50], Loss: 3.4106
Epoch [1/50], Loss: 3.5063
Epoch [1/50], Loss: 3.8523
Epoch [1/50], Loss: 3.2214
Epoch [1/50], Loss: 2.6263
Epoch [1/50], Loss: 3.0389
Epoch [1/50], Loss: 3.9064
Epoch [1/50], Loss: 3.1807
Epoch [1/50], Loss: 3.4979
Epoch [1/50], Loss: 2.6460
Epoch [1/50], Loss: 2.7840
Epoch [1/50], Loss: 4.3480
Epoch [1/50], Loss: 2.4595
Epoch [1/50], Loss: 3.2974


KeyboardInterrupt: 

In [40]:
for epoch in range(num_epochs):
    permutation = torch.randperm(training_set.size()[0])
    for i in range(0, training_set.size()[0], batch_size):
        opt.zero_grad()
        indices = permutation[i:i + batch_size]
        batch_features = training_set[indices]
        batch_features = batch_features.reshape(-1, 300).to(device)
        batch_labels = torch.LongTensor([label for label, sent in train[indices]])
        batch_features = batch_features.reshape(-1, 1).to(device)
        batch_outputs = model(batch_features)
        loss = loss_function(batch_outputs, batch_labels)
        print(loss)
        loss.backward()
        opt.step()
    

TypeError: cannot unpack non-iterable int object

In [ ]:
# cdata=[]
# target = 
# for l,d in b:
#     bow_vec = make_bow_vector(d, indexed_vocab)
#     cdata.append(list(bow_vec.tolist()))
#     target.append(labels[l])
# c_tensor= torch.FloatTensor(cdata)
# t_tensor=torch.LongTensor(target)
# y_pred = bow(c_tensor)
# loss = loss_function(y_pred,t_tensor)
# loss.backward()
# opt.step()

In [ ]:
#testing
test_len =len(test)
correct=0
for label,data in test:
    bow_vec = make_bow_vector(data, word2idx)
    logprobs = bow(bow_vec)
    print(logprobs)
    pred = np.argmax(logprobs.data.numpy())
    if pred==label:
        correct+=1
    print('prediction: {}'.format(pred))
    print('actual: {}'.format(label))
accuracy = correct/test_len
print('accuracy: {}'.format(accuracy))

In [83]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Hyper-parameters 
input_size = 300
hidden_size = 500
num_classes = 51
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

# Train the model
permutation = torch.randperm(len(train))
for epoch in range(num_epochs):
    i = 0
    for label, question in train[permutation]:
        i += 1
        # Move tensors to the configured device
        bow_vec = make_bow_vector(question, word2idx)
        bow_vec = bow_vec.reshape(-1, 300).to(device)
        label = torch.LongTensor([label])
        label = label.to(device)
        # Forward pass
        output = model(bow_vec)
        loss = criterion(output, label)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, loss.item()))

Epoch [1/5], Loss: 1.5975
Epoch [1/5], Loss: 3.9109
Epoch [1/5], Loss: 2.0692
Epoch [1/5], Loss: 0.6898
Epoch [1/5], Loss: 1.0542
Epoch [1/5], Loss: 3.6905
Epoch [1/5], Loss: 4.1781
Epoch [1/5], Loss: 3.9456
Epoch [1/5], Loss: 0.3858
Epoch [1/5], Loss: 3.0947
Epoch [1/5], Loss: 2.3612
Epoch [1/5], Loss: 1.4534
Epoch [1/5], Loss: 3.8365
Epoch [1/5], Loss: 0.3030
Epoch [1/5], Loss: 3.4972
Epoch [1/5], Loss: 0.2402
Epoch [1/5], Loss: 0.1577
Epoch [1/5], Loss: 0.1685
Epoch [1/5], Loss: 0.8692
Epoch [1/5], Loss: 0.2664
Epoch [1/5], Loss: 2.8854
Epoch [1/5], Loss: 0.0023
Epoch [1/5], Loss: 2.0450
Epoch [1/5], Loss: 1.0335
Epoch [1/5], Loss: 2.6692
Epoch [1/5], Loss: 2.1869
Epoch [1/5], Loss: 0.4868
Epoch [1/5], Loss: 2.3914
Epoch [1/5], Loss: 0.2144
Epoch [1/5], Loss: 0.8490
Epoch [1/5], Loss: 2.0523
Epoch [1/5], Loss: 1.3947
Epoch [1/5], Loss: 2.7750
Epoch [1/5], Loss: 0.0199
Epoch [1/5], Loss: 2.7137
Epoch [1/5], Loss: 0.8689
Epoch [1/5], Loss: 1.4676
Epoch [1/5], Loss: 0.5477
Epoch [1/5],

KeyboardInterrupt: 

In [ ]:
for i, m in train_dataset:
    print(m)

In [39]:
labels = labels.to(devic#e)

In [40]:
labels

tensor([3], device='cuda:0')

In [5]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 1
learning_rate = 0.001

# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 28 * 28).to(device)
        print(images)
        labels = labels.to(device)
        print(labels)
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0

         0.0000]], device='cuda:0')
tensor([4], device='cuda:0')
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0667,
         0.3255, 0.9255, 0.7529, 0.0000, 0.0

         0.0000]], device='cuda:0')
tensor([2], device='cuda:0')
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0

         0.0000]], device='cuda:0')
tensor([5], device='cuda:0')
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0

         0.0000]], device='cuda:0')
tensor([2], device='cuda:0')
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0

...

         0.0000]], device='cuda:0')
tensor([3], device='cuda:0')
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0

         0.0000]], device='cuda:0')
tensor([0], device='cuda:0')
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.2510, 0.7490, 1.0000, 0.7490, 0.5020, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0

         0.0000]], device='cuda:0')
tensor([5], device='cuda:0')
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0

         0.0000]], device='cuda:0')
tensor([5], device='cuda:0')
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0

KeyboardInterrupt: 

In [76]:
for label, question in train[permutation]:
    print(question)

['how', 'clean', 'badly', 'tarnished', 'brass', '?']
['how', 'many', 'dollars', 'day', 'arthur', 'frommer', 'say', 'could', 'get', 'europe', '1968', '?']
['aspartame', 'known', 'what', 'name', '?']
['what', 'scandinavian', 'country', 'covers', '173', ',', '732', 'square', 'miles', '?']
['what', 'movie', 'madilyn', 'kahn', 'star', 'gene', 'wilder', '?']
['what', 'qualifications', 'individuals', 'donating', 'blood', '?']
['what', 'wheel', 'blaise', 'pascal', 'invent', 'search', 'perpetual', 'motion', '?']
['what', "'s", 'east', 'european', 'country', 'tied', 'ruble', '?']
['who', 'nicknamed', 'little', 'corporal', '?']
['what', 'geoscientist', '?']
['how', 'make', 'dumplings', '?']
['what', 'long', 'march', 'china', '?']
['what', 'country', 'surrounds', 'san', 'marino', ',', 'world', "'s", 'smallest', 'republic', '?']
['what', 'dextropropoxyphen', 'napsylate', '?']
['what', 'tv', 'detective', 'craig', 'stevens', 'play', '?']
['what', 'robert', 'louis', 'stevenson', 'novel', 'inspired', '

['where', 'guinea', 'pigs', 'originate', '?']
['what', 'current', 'unemployment', 'rate', 'u.s.', '?']
['why', 'say', 'children', "'s", 'cough', 'syrup', 'drive', 'using', 'medication', '?']
['what', 'year', 'first', 'patent', 'pull-tab', 'opener', 'cans', 'obtained', '?']
['what', 'deltiologist', 'collect', '?']
['name', 'south', 'african', 'diamond', 'producer', '?']
['who', 'columbia', 'pictures', 'head', 'who', 'forged', 'actor', 'cliff', 'robertson', "'s", 'name', '$1', ',', '000', 'check', '?']
['what', 'texas', 'surgeon', 'performed', 'first', 'artificial', 'heart', 'transplant', '?']
['how', 'many', 'url', 'extensions', '?', 'what', '?']
['what', 'stephen', 'sondheim', 'ballad', 'ends', ':', '``', 'well', ',', 'maybe', 'next', 'year..', '.', "''", '?']
['how', 'measure', 'heat', 'sun', '?']
['what', 'definition', 'cascade', '?']
['what', 'powers', 'weaknesses', 'green', 'lantern', "'s", 'ring', '?']
['where', '49', 'steps', '?']
['what', 'origin', 'word', ',', 'jj', '.']
['kent

['what', 'substance', 'gives', 'food', '``', 'taste', "''", 'when', 'eaten', '?']
['what', 'method', 'called', '``', 'cold', 'fusion', "''", 'create', '?']
['what', 'grenada', "'s", 'main', 'commodity', 'export', '?']
['what', 'president', 'hit', 'jogging', 'paths', 'enhance', 'athletic', 'image', ',', 'sporting', 'no.', '39', ',', 'almost', 'collapsed', 'road', 'race', '?']
['what', 'country', 'ancient', 'romans', 'refer', 'hibernia', '?']
['what', 'courier', '?']
['what', 'part', 'benjamin', 'franklin', 'play', 'development', 'newspaper', 'america', '?']
['what', 'life', 'expectancy', 'elephant', '?']
['how', 'websites', 'like', 'yahoo', 'excite', 'make', 'money', '?']
['what', 'web', 'sites', 'linked', 'report', 'genesis', 'eldercare', '?']
['what', 'kind', 'education', 'would', 'need', 'become', 'athletic', 'trainer', 'nfl', '?']
['what', 'oldtime', 'kids', "'", 'fare', 'tv', 'guide', 'writer', 'jeff', 'greenfield', 'call', '``', 'one', 'most...subversive', 'tv', 'shows', 'american

['what', 'famed', 'strip', 'land', '15-minute', 'boat', 'trip', 'across', 'venetian', 'lagoon', 'venice', '?']
['what', 'king', 'boycotted', 'prince', 'charles', "'s", 'wedding', '?']
['how', 'use', 'intranet', '?']
['what', 'toy', 'make', 'sleep', '?']
['what', 'origin', 'first', 'name', '`', 'breony', "'", '?']
['how', 'many', 'bytes', 'terabyte', '?']
['what', 'taj', 'majal', 'made', '?']
['where', 'erykah', 'badu', 'originally', '?']
['what', 'definition', 'cecum', '?']
['what', 'largest', 'sculpture', 'world', '?']
['how', 'long', 'ago', 'roe', 'vs.', 'wade', 'decision', 'supreme', 'court', '?']
['what', 'spy', 'novelist', 'served', 'moscow', 'correspondent', 'reuter', 'times', 'london', '?']
['what', 'average', 'salary', 'paleontologist', '?']
['what', 'ill-fated', 'american', 'general', 'dragged', 'bull', 'terrier', 'named', 'willie', 'world', 'war', 'ii', '?']
['what', 'phone', 'number', 'call', 'tree', 'planted', '?']
['what', 'technique', 'popularly', 'used', 'detect', 'birth

['what', 'russian', 'master', 'spy', 'lived', 'u.s.', 'name', 'emil', 'goldfus', '?']
['what', 'april', "'s", 'gemstone', '?']
['what', 'final', 'four', 'refer', 'sports', 'world', '?']
['what', 'fowl', 'grabs', 'spotlight', 'chinese', 'year', 'monkey', '?']
['where', 'find', 'case', 'individuals', 'disabilities', 'education', 'act', '1991', '?']
['what', 'former', 'major-league', 'left-handed', 'baseball', 'pitcher', 'known', '``', 'space', 'man', "''", '?']
['how', 'tall', 'replica', 'matterhorn', 'disneyland', '?']
['which', 'tennessee', 'williams', 'play', 'one', 'characters', 'killed', 'devoured', 'mob', 'starving', 'children', '?']
['where', 'rider', 'college', 'located', '?']
['how', 'many', 'referees', 'work', 'soccer', 'game', '?']
['what', 'vdrl', 'test', 'blood', '?']
['how', 'many', 'times', 'nickel-cadmium', 'rechargeable', 'battery', 'recharged', '?']
['where', 'oldest', 'living', 'thing', 'earth', '?']
['who', 'inventor', 'silly', 'putty', '?']
['how', 'asia', 'get', 'na

...

['what', 'homelite', 'inc.', 'home', 'page', '?']
['what', 'saltpeter', '?']
['what', 'type', 'animal', 'easiest', 'kept', 'pet', '?']
['what', 'brand', 'jeans', 'calvin', 'klein', 'admit', '1984', 'interview', 'felt', 'comfortable', 'wearing', '?']
['what', 'african', 'capital', 'named', 'u.s.', 'president', '?']
['who', 'said', ':', '``', 'soldiers', 'win', 'battles', 'generals', 'get', 'credit', "''", '?']
['what', 'country', 'worlds', 'leading', 'supplier', 'cannabis', '?']
['how', 'general', 'mills', 'manufacture', 'cheerios', '?']
['why', 'several', 'san', 'diego', 'schools', 'stop', 'serving', 'apples', 'students', '?']
['who', 'star', '1965', 'broadway', 'hit', 'golden', 'boy', '?']
['what', 'corporation', 'madonna', 'advertise', '?']
['what', 'crooner', 'joined', 'andrews', 'sisters', 'pistol', 'packin', 'mama', '?']
['define', 'pheonix', 'club', '?']
['why', 'rooftops', 'give', 'steam', 'big', 'rainstorm', '?']
['what', 'origin', 'peace', 'symbol', '?']
['how', 'scientific', 

['which', 'real', 'berry', ':', 'blackberry', ',', 'cranberry', ',', 'raspberry', ',', 'strawberry', '?']
['what', 'group', 'included', 'abbey', 'hoffman', ',', 'jerry', 'rubin', ',', 'tom', 'hayden', '?']
['who', 'won', 'nobel', 'peace', 'prize', '1991', '?']
['how', 'tall', 'animated', 'frankenstein']
['where', 'woodstock', 'take', 'place', '?']
['what', 'alloy', 'copper', 'tin', 'form', '?']
['where', 'rocks', 'come', '?']
['what', 'natural', 'attractions', 'draw', 'visitors', 'united', 'states', '?']
['how', 'old', 'gene', 'siskel', '?']
['where', 'find', 'pictorial', 'directions', 'how', 'build', 'simple', 'treehouse', '?']
['how', 'long', 'dog', 'sleep', '?']
['what', 'biologist', '?']
['what', 'california', 'desert', 'dubbed', 'high', 'desert', '?']
['cnn', "'s", 'first', 'broadcast', 'occurred', 'what', 'date', '?']
['what', 'temperature', 'today', '?']
['what', 'leukemia', '?']
['what', 'know', 'multicultural', 'multilingual', 'schools', '?']
['what', 'country', 'offered', 'al

['what', 'substance', 'joseph', 'priestley', 'name', 'ability', 'erase', 'pencil', 'marks', '?']
['what', 'habitat', 'chickadee', '?']
['what', 'hockey', 'player', 'ronald', 'reagan', 'joke', 'would', 'swap', 'texas', '?']
['who', 'shakespeare', "'s", 'moorish', 'general', '?']
['what', 'attorney-general', 'ordered', 'closing', 'alcatraz', '?']
['what', 'meant', 'term', '``', 'yield', 'maturity', "''", 'reference', 'bonds', '?']
['how', 'say', 'love', 'spanish', '?']
['what', 'chromatology', '?']
['what', 'goldfish', 'lose', 'kept', 'dimly-lit', 'running', 'water', '?']
['what', 'articles', 'clothing', 'tokens', 'monopoly', '?']
['what', 'largest', 'city', 'great', 'lakes', '?']
['who', 'leaders', 'byzantine', 'empire', '?']
['who', 'thucydides', '?']
['how', 'many', 'people', 'died', 'south', 'carolina', 'highways', '1998', '?']
['how', 'many', 'people', 'captain', 'america', '?']
['how', 'many', 'spears', 'kenya', "'s", 'flag', '?']
['what', 'time', 'takes', 'typist', 'type', 'screen

['what', 'south', 'american', 'country', 'won', 'first', 'world', 'cup', 'soccer', 'title', '1978', '?']
['what', 'leper', '?']
['what', 'acronym', 'national', 'bureau', 'investigation', '?']
['why', 'called', '``', 'hamburger', "''", 'ham', '?']
['when', 'president', 'kennedy', ',', 'lee', 'harvey', 'oswald', ',', 'jack', 'ruby', 'die', '?']
['what', 'country', 'allowed', 'drive', 'israel', '1979', '?']
['who', 'advised', 'listeners', '``', 'see', 'u.s.a.', 'chevrolet', "''", '?']
['how', 'many', 'species', 'sharks', '?']
['name', 'randy', 'craft', "'s", 'lawyer', '.']
['who', 'won', 'two', 'gold', 'medals', 'skiing', 'olympic', 'games', 'calgary', '?']
['what', 'origin', 'ethnic', 'category', '``', 'caucasian', "''", '?']
['who', 'last', 'woman', 'executed', 'england', '?']
['how', 'much', 'cost', 'tree', 'planted', 'dialing', ',', '900', ',', '740-tree', '?']
['what', 'name', '1899', 'policy', 'forced', 'china', 'which', 'nations', 'equal', 'trading', 'rights', '?']
['which', 'class

['when', 'thai', 'new', 'year', '?']
['what', 'determines', 'value', 'pearl', '?']
['where', 'union', 'stockyards', '?']
['how', 'much', 'stronger', 'new', 'vitreous', 'carbon', 'material', 'invented', 'tokyo', 'institute', 'technology', 'compared', 'material', 'made', 'cellulose', '?']
['what', 'stefan', 'edberg', "'s", 'native', 'country', '?']
['what', 'sea', 'bombay', '?']
['how', 'one', 'correctly', 'pronounce', '`', 'qigong', "'", '?']
['what', 'largest', 'expensive', 'freeway', 'construction', 'project', 'u.s.', 'right', '?']
['what', 'four', 'tournaments', 'make', 'tennis', "'", 'grand', 'slam', '?']
['what', 'temperature', 'baking', 'peachy', 'oat', 'muffins', '?']
['where', 'ocho', 'rios', '?']
['what', 'two', 'us', 'biochemists', 'won', 'nobel', 'prize', 'medicine', '1992', '?']
['what', 'yohimbine', '?']
['what', 'abbreviation', 'micro', '?']
['what', 'city', 'which', 'maurizio', 'pellegrin', 'lives', 'called', '?']
['which', 'two', 'products', 'use', 'tiger', 'symbol', '?'

In [111]:
a = torch.tensor([11,2,3,3])
a.reshape(-1, 4)

tensor([[11,  2,  3,  3]])

In [43]:
for label, sent in train[indices]:
    print(label)

TypeError: cannot unpack non-iterable int object

In [41]:
train[indices]

array([6, list(['how', 'tall', 'animated', 'frankenstein'])], dtype=object)